## Setup imports

In [1]:
import sys
import os
import logging
import numpy as np
import torch
import monai.networks.nets as nets
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    CropForegroundd,
    ToTensord,
    RandAxisFlipd,
    RandAffined,
    SpatialPadd,
    Activationsd,
    Resized,
    RandGaussianNoised,
)
from transforms import (
    CTWindowd,
    #RandCTWindowd,
    CTSegmentation,
    RelativeCropZd,
)
from monai.data import DataLoader, Dataset, PersistentDataset, CacheDataset
from monai.transforms.croppad.batch import PadListDataCollate
from monai.utils import NumpyPadMode, set_determinism
from monai.utils.enums import Method
from monai.config import print_config
from sklearn.model_selection import train_test_split
from trainer import Trainer
from validator import Validator
from utils import (
    setup_directories,
    create_device,
    get_data_from_info,
    large_image_splitter,
    calculate_class_imbalance,
    balance_training_data,
)
from test_data_loader import TestDataset
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


MONAI version: 0.5.0rc5+2.gc3dbb8a
Numpy version: 1.19.5
Pytorch version: 1.7.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: c3dbb8a46c3d38edeff65b33f287eb3fc1bd9f9d

Optional dependencies:
Pytorch Ignite version: 0.4.4
Nibabel version: 3.2.1
scikit-image version: 0.18.1
Pillow version: 8.1.2
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.8.2
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.59.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup directories

In [2]:
dirs = setup_directories()

## Setup torch device

In [3]:
# pass "cuda" to use the GPU
device, using_gpu = create_device("cuda")

Cuda device is not supported, switching to CPU


## Load and randomize images

In [5]:
# HACKATON image and segmentation data
hackathon_dir = os.path.join(dirs["data"], 'HACKATHON')
with open(os.path.join(hackathon_dir, "train.txt"), 'r') as fp:
    train_info_hackathon = [entry.strip().split(',') for entry in fp.readlines()]
image_dir = os.path.join(hackathon_dir, 'images', 'train')
seg_dir = os.path.join(hackathon_dir, 'segmentations', 'train')
_train_data_hackathon = get_data_from_info(image_dir, seg_dir, train_info_hackathon, dual_output=False)
large_image_splitter(_train_data_hackathon, dirs["cache"])
balance_training_data(_train_data_hackathon)
# PSUF data
"""psuf_dir = os.path.join(dirs["data"], 'psuf')
with open(os.path.join(psuf_dir, "train.txt"), 'r') as fp:
    train_info = [entry.strip().split(',') for entry in fp.readlines()]
image_dir = os.path.join(psuf_dir, 'images')
train_data_psuf = get_data_from_info(image_dir, None, train_info)"""
# Split data into train, validate and test
train_split, test_data_hackathon = train_test_split(_train_data_hackathon, test_size=0.2, shuffle=True, random_state=42)
train_data_hackathon, valid_data_hackathon = train_test_split(train_split, test_size=0.2, shuffle=True, random_state=43)

Splitting large images...
original data len: 355
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0001.nii.gz into 4 parts shape: (512, 512, 326)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0022.nii.gz into 3 parts shape: (512, 512, 276)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0011.nii.gz into 3 parts shape: (512, 512, 296)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0016.nii.gz into 4 parts shape: (512, 512, 361)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0008.nii.gz into 3 parts shape: (512, 512, 261)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0015.nii.gz into 3 parts shape: (512, 512, 306)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0021.nii.gz into 4 parts shape: (512, 512, 361)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/0009.nii.gz into 3 parts shape: (512, 512, 306)
splitting image: /home/jupyter/RIS/data/HACKATHON/images/train/

## Setup transforms

In [6]:
# Crop foreground
crop_foreground = CropForegroundd(
    keys=["image"],
    source_key="image",
    margin=(5, 5, 0),
    #select_fn = lambda x: x != 0
)
# Crop Z
crop_z = RelativeCropZd(keys=["image"], relative_z_roi=(0.05, 0.15))
# Window width and level (window center)
WW, WL = 1500, -600
ct_window = CTWindowd(keys=["image"], width=WW, level=WL)
# Random variatnon in CT window
"""rand_WW, rand_WL = 50, 25
rand_ct_window = RandCTWindowd(
    keys=["image"],
    prob=1.0,
    width=(WW-rand_WW, WW+rand_WW),
    level=(rand_WL-25, rand_WL+25)
)"""
# Random axis flip
rand_axis_flip = RandAxisFlipd(keys=["image"], prob=0.1)
# Rand affine transform
rand_affine = RandAffined(
    keys=["image"],
    prob=0.50,
    rotate_range=(0, 0, np.pi/16),
    shear_range=(0.05, 0.05, 0.0),
    translate_range=(0, 0, 0),
    scale_range=(0.05, 0.05, 0.0),
    padding_mode="zeros"
)
# Pad image to have hight at least 30
spatial_pad = SpatialPadd(keys=["image"], spatial_size=(-1, -1, 30))
# Resize image x and y
resize_fator = 0.5
resize = Resized(keys=["image"], spatial_size=(int(512*resize_fator), int(512*resize_fator), -1), mode="trilinear")
rand_gaussian_noise = RandGaussianNoised(keys=["image"], prob=0.25)

#### Create transforms

In [7]:
common_transform = Compose([
    LoadImaged(keys=["image"]),
    ct_window,
    CTSegmentation(keys=["image"]),
    AddChanneld(keys=["image"]),
    resize,
    crop_foreground,
    crop_z,
    spatial_pad,
])
hackathon_train_transform = Compose([
    common_transform,
    rand_axis_flip,
    rand_affine,
    rand_gaussian_noise,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_valid_transfrom = Compose([
    common_transform,
    ToTensord(keys=["image"]),
]).flatten()
psuf_transforms = Compose([
    LoadImaged(keys=["image"]),
    AddChanneld(keys=["image"]),
    ToTensord(keys=["image"]),
])

## Setup data

In [8]:
#set_determinism(seed=100)
train_dataset = PersistentDataset(data=train_data_hackathon[:], transform=hackathon_train_transform, cache_dir=dirs["persistent"])
valid_dataset = PersistentDataset(data=valid_data_hackathon[:], transform=hackathon_valid_transfrom, cache_dir=dirs["persistent"])
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    pin_memory=using_gpu,
    num_workers=1,
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=2,
    shuffle=True,
    pin_memory=using_gpu,
    num_workers=1,
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)

## Setup network, loss function, optimizer and scheduler

In [9]:
network = nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=1).to(device)
# pos_weight for class imbalance
pos_weight = calculate_class_imbalance(train_info_hackathon).to(device)
loss_function = torch.nn.BCEWithLogitsLoss(pos_weight)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-5)#, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95, last_epoch=-1)


## Setup validator and trainer

In [10]:
valid_post_transforms = Compose([
    Activationsd(keys="pred", sigmoid=True),
])
validator = Validator(
    device=device,
    val_data_loader=valid_loader,
    network=network,
    post_transform=valid_post_transforms,
    amp=using_gpu,
    non_blocking=using_gpu
)

trainer = Trainer(
    device=device,
    out_dir=dirs["out"],
    out_name="DenseNet121",
    max_epochs=120,
    train_data_loader=train_loader,
    network=network,
    optimizer=optimizer,
    loss_function=loss_function,
    lr_scheduler=None,
    validator=validator,
    amp=using_gpu,
    non_blocking=using_gpu
)

## Run trainer

In [11]:
trainer.run()

Training started: 13/04/2021 08:41:09
INFO:ignite.engine.engine.Trainer:Engine run resuming from iteration 0, epoch 0 until 120 epochs


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 1/203 -- loss: 3.1009 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 2/203 -- loss: 3.0832 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 3/203 -- loss: 3.0350 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 4/203 -- loss: 3.4089 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 5/203 -- loss: 3.1821 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 6/203 -- loss: 3.1963 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 7/203 -- loss: 2.6491 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 8/203 -- loss: 2.6568 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 9/203 -- loss: 2.9314 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 10/203 -- loss: 2.6882 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 11/203 -- loss: 2.6365 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 12/203 -- loss: 3.1590 
INFO:ignite.engine.engine.Trainer:Epoch: 1/120, Iter: 13/203 -- loss: 3.4

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


ERROR:ignite.engine.engine.Validator:Current run is terminating due to exception: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/monai/data/utils.py", line 264, in list_data_collate
    ret[k] = default_collate([d[k] for d in data])
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 81, in default_collate
    raise RuntimeError('each element in list of batch should be of equal size')
RuntimeError: each element in list of batch should be of equal size

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/opt/conda/lib/python3.7

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/monai/data/utils.py", line 264, in list_data_collate
    ret[k] = default_collate([d[k] for d in data])
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 81, in default_collate
    raise RuntimeError('each element in list of batch should be of equal size')
RuntimeError: each element in list of batch should be of equal size

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/opt/conda/lib/python3.7/site-packages/monai/transforms/croppad/batch.py", line 113, in __ca